In [29]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Models and selection methods
from sklearn.base import clone
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
# Binary classifier metrics
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
# Linear regression metrics
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
#Pré-Processamento de dados
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectFromModel

In [4]:
# Estatisticas para classificadores
def printClassResults(truth, preds):
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds))
    print("The Recall is: %7.4f" % recall_score(truth, preds))
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(truth, preds))
    print()
    print("This is the Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(truth, preds)))

# Plot das previsoes vs dados reais
def displayPlot(preds, truth):
    plt.figure(figsize=(5,5))
    plt.scatter(preds, truth)
    plt.axline((0, 0), slope=1, color="red")
    plt.grid()
    plt.show()

## Pré-Processamento dos dados
Preparação do dataset - importação, normalização e preenchimento dos missing values

In [5]:
#Criar dataframe
bio_a = pd.read_csv('biodegradable_a.csv')
#Separação das 41 variáveis do y
X_bio_a=bio_a.drop(columns=["Biodegradable"])
y_bio_a=bio_a['Biodegradable']
#Converter para numpy array
Xc_bio= X_bio_a.to_numpy()
yc_bio= y_bio_a.to_numpy()
print(Xc_bio)
# Divisão do dataset em training set e independent validation set
X_bio_train, X_bio_test, y_bio_train, y_bio_test = train_test_split(Xc_bio, yc_bio, test_size=0.25, random_state=512)

[[3.919      2.6909     0.         ... 7.253      0.         0.        ]
 [4.17       2.1144     0.         ... 7.257      0.         0.        ]
 [3.932      3.2512     0.         ... 7.601      0.         0.        ]
 ...
 [4.29477099 3.47122594 0.         ... 7.69932352 0.         0.        ]
 [4.56037572 3.89639031 0.         ... 7.90880199 0.         0.        ]
 [4.04509735 2.84718513 0.         ... 6.86331402 0.         0.        ]]


Passamos agora à normalização dos dados. Vão ser escolhidos os seguintes métodos de normalização para comparar mais tarde: MinMax Scaler, Standard Scaler e Power Transformer

In [16]:
#Passemos à normalização dos dados
##NOTA: A partir daqui é só experiência (só para se poder continuar código)
scaler_p = PowerTransformer()
scaler_st = StandardScaler()
scaler_min= MinMaxScaler()

#Transformar dados
X_bio_train_p=scaler_p.fit_transform(X_bio_train)
X_bio_test_p=scaler_p.fit_transform(X_bio_test)

X_bio_train_st=scaler_st.fit_transform(X_bio_train)
X_bio_test_st=scaler_st.fit_transform(X_bio_test)

X_bio_train_min=scaler_min.fit_transform(X_bio_train)
X_bio_test_min=scaler_min.fit_transform(X_bio_test)

print(pd.DataFrame(X_bio_train_p))

            0         1         2         3         4         5         6   \
0    -1.857579  0.168452 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
1    -1.854459 -1.019003 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
2     0.430395  1.193057 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
3     0.421222 -0.036376 -0.298474 -0.092755 -0.329914 -0.193061  1.293260   
4    -0.751657 -1.775834 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
...        ...       ...       ...       ...       ...       ...       ...   
3418  1.664765  0.591478  3.349907 -0.092755 -0.329914 -0.193061  1.293260   
3419  0.244807 -0.641362 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
3420 -0.562774  0.804821 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
3421  0.755840 -0.060657 -0.298474 -0.092755 -0.329914 -0.193061  1.293260   
3422  1.054821 -0.193085 -0.298474       NaN -0.329914 -0.193061  1.293260   

            7         8         9   ...        31        32    

Com os dados normalizados, podemos passar à imputação dos valores em falta. Escolheu-se o método de imputação utilizando o K-Nearest Neighbours em todos 

In [25]:
#Tratamento dos Missing values -> Utilizar Imputação de KNN
imputer = KNNImputer(n_neighbors=3, weights="uniform")

X_bio_train_p=imputer.fit_transform(X_bio_train_p)
X_bio_test_p=imputer.fit_transform(X_bio_test_p)

X_bio_train_st=imputer.fit_transform(X_bio_train_st)
X_bio_test_st=imputer.fit_transform(X_bio_test_st)

X_bio_train_min=imputer.fit_transform(X_bio_train_min)
X_bio_test_min=imputer.fit_transform(X_bio_test_min)

print(pd.DataFrame(X_bio_train_p))

            0         1         2         3         4         5         6   \
0    -1.857579  0.168452 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
1    -1.854459 -1.019003 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
2     0.430395  1.193057 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
3     0.421222 -0.036376 -0.298474 -0.092755 -0.329914 -0.193061  1.293260   
4    -0.751657 -1.775834 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
...        ...       ...       ...       ...       ...       ...       ...   
3418  1.664765  0.591478  3.349907 -0.092755 -0.329914 -0.193061  1.293260   
3419  0.244807 -0.641362 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
3420 -0.562774  0.804821 -0.298474 -0.092755 -0.329914 -0.193061 -0.766057   
3421  0.755840 -0.060657 -0.298474 -0.092755 -0.329914 -0.193061  1.293260   
3422  1.054821 -0.193085 -0.298474 -0.092755 -0.329914 -0.193061  1.293260   

            7         8         9   ...        31        32    

Resta apenas ver quais são as variáveis mais relevantes. Para tal, vamos utilizar o SelectFromModel do scikit-learn utilizando RandomForests.

In [34]:
rfr=RandomForestClassifier(random_state=45)
sel = SelectFromModel(estimator=rfr, threshold=0.1)